!pip install -U scikit-learn

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# The code was removed by Watson Studio for sharing.

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,pub_rec_bankruptcies,tax_liens,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,CHARGED_OFF
0,1,5000.0,5000.0,4975.0,36.0,10.65,162.87,B,B2,NaN,...,0.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2,2500.0,2500.0,2500.0,60.0,15.27,59.83,C,C4,Ryder,...,0.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,1


In [3]:
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

### No point of evaluating loans with loan_amnt or  null

In [4]:
loans_raw.dropna(subset=['loan_amnt', 'total_acc'], inplace=True)

### List missing values aggregation

In [5]:
missing_values_table(loans_raw)

Your selected dataframe has 54 columns.
There are 18 columns that have missing values.


,Missing Values,% of Total Values
settlement_term,42346,99.6
debt_settlement_flag_date,42346,99.6
settlement_percentage,42346,99.6
settlement_amount,42346,99.6
settlement_date,42346,99.6
settlement_status,42346,99.6
next_pymnt_d,39786,93.6
mths_since_last_record,38855,91.4
mths_since_last_delinq,26897,63.3
desc,13292,31.3


### Drop variables with too many nulls

In [6]:
too_many_na = ["settlement_term", "settlement_percentage", "settlement_amount", "settlement_date",  "settlement_status", "debt_settlement_flag_date", "next_pymnt_d", "desc", "mths_since_last_record", "mths_since_last_delinq"]
loans_raw.drop(columns=too_many_na, inplace=True)


In [7]:
missing_values_table(loans_raw)

Your selected dataframe has 44 columns.
There are 8 columns that have missing values.


,Missing Values,% of Total Values
emp_title,2614,6.1
pub_rec_bankruptcies,1336,3.1
emp_length,1112,2.6
last_pymnt_d,83,0.2
tax_liens,76,0.2
revol_util,61,0.1
title,13,0.0
last_credit_pull_d,3,0.0


In [8]:
from sklearn.model_selection import train_test_split
credits_train, credits_test = train_test_split(loans_raw, test_size=0.2, random_state=42)

In [9]:
credits_target = credits_train["CHARGED_OFF"].copy()
credits_train = credits_train.drop("CHARGED_OFF", axis=1)

In [10]:
credits_num = credits_train.select_dtypes(exclude=['object'])
credits_cat = credits_train.select_dtypes(include=['object'])
num_attribs = credits_num.columns
cat_attribs = credits_cat.columns

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="most_frequent")),
        ('oh_encoder', OneHotEncoder()),
    ])

In [12]:
from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

credits_prepared = full_pipeline.fit_transform(credits_train)


In [13]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier()
rf_classifier.fit(credits_prepared, credits_target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [14]:
from sklearn.model_selection import cross_val_score
import numpy as np

print (np.mean(cross_val_score(rf_classifier, credits_prepared, credits_target, cv=10)))

0.975443987063
